### Подключение библиотек. Загрузка и просмотр данных. Предобработка, очистка данных

In [806]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [808]:
df = pd.read_csv('events filters к тестовому Product Analyst Q1 2024.csv')

In [810]:
df.head(10)

,event_category,event_action,event_label,total_events,unique_events
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2
5,city_landing,clear_filter_mobile,Vienna,12,11
6,city_landing,start_date_click,Vilnius,9,4
7,city_landing,ticket-type_checkbox,Irkutsk / Мини-группа,29,26
8,city_landing,price_button_submit,Vladivostok / Показать предложения (28),3,3
9,city_landing,filters-categories_click,Krakow / Необычные,3,3


### Произведем очистку данных, выделим данные из event_label на колонки города и текста, удалим ненужные данные

In [813]:
df['city'] = df.event_label.str.split('/').str[0] #колонка с городами
df['text'] = df.event_label.str.split('/').str[1] #колонка с дополнительными данными 
df['text'] = df['text'].str.split('(').str[0] #уберем ненужные цифры
for i, city in enumerate(df['city']): #исправим наличие пробела в некоторых городах в конце строки
    if city[-1] == ' ':
        df.at[i, 'city'] = city[:-1]
df.head(10)

,event_category,event_action,event_label,total_events,unique_events,city,text
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7,Tula,Показать предложения
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10,Sochi,Показать предложения
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2,Penza,Сортировка
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1,Gelendzhik,ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2,Moscow,Показать предложения
5,city_landing,clear_filter_mobile,Vienna,12,11,Vienna,NaN
6,city_landing,start_date_click,Vilnius,9,4,Vilnius,NaN
7,city_landing,ticket-type_checkbox,Irkutsk / Мини-группа,29,26,Irkutsk,Мини-группа
8,city_landing,price_button_submit,Vladivostok / Показать предложения (28),3,3,Vladivostok,Показать предложения
9,city_landing,filters-categories_click,Krakow / Необычные,3,3,Krakow,Необычные


In [815]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19092 entries, 0 to 19091
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   event_category  19092 non-null  object
 1   event_action    19092 non-null  object
 2   event_label     19092 non-null  object
 3   total_events    19092 non-null  int64 
 4   unique_events   19092 non-null  int64 
 5   city            19092 non-null  object
 6   text            17013 non-null  object
dtypes: int64(2), object(5)
memory usage: 1.0+ MB


### П. 1, 2 Посчитаем сколько людей пользуются фильтрами по отдельным городам и суммарно по всем городам
Применение фильтра будем считать по event_action - price_button_submit, применению фильтров.
Количество будем считать как unique_events - уникальные ивенты на сайте

In [818]:
counts_all_cities = df.unique_events[df['event_action'].isin(['price_button_submit'])].sum()
counts_city = df[df['event_action'].isin(['price_button_submit'])].groupby(['city'])[['unique_events', 'total_events']].sum().reset_index()
counts_city = counts_city.sort_values(by=['unique_events'], ascending=False)

pd.concat([counts_city.head(15), counts_city.tail(15)]) #первые 15 и  последние 15 городов по использованию фильтров

,city,unique_events,total_events
132,Kaliningrad,5222,6113
262,Saint Petersburg,5028,5601
197,Moscow,3176,3598
286,Sochi,2909,3286
124,Istanbul,2357,2778
81,Dubai,2095,2406
53,Budapest,1843,2154
139,Kazan,1839,2072
13,Anapa,1619,1837
278,Sharm-el-Sheikh,1440,1661


In [820]:
counts_all_cities

np.int64(63063)

1. Таким образом, мы видим, что общее количество использования фильтров - 63063

2. Самые популярные города:
- Калининград
- Санкт-Петербург
- Москва
- Сочи
- Стамбул
  
  Самые непопулярные города где использовался фильтр:
- Тоскана
- Тольятти
- Свияжск
- Таруса
- Бетлегем

И другие города. 
Кроме того, есть большой список городов, где фильтрами не пользовались вообще:

In [823]:
#df[~df.city.isin(counts_city['city'])].sort_values(by='city').city.unique()
df[~df['city'].isin(counts_city['city'])]['city'].unique()

array(['Hertogenbosch', 'Costa Dorada', 'Versailles', 'Leningrad Oblast',
       'Jakarta', 'Uganda', 'Bilbao', 'Tartu', 'Dombay', 'Nantes',
       'Syracuse', 'Netanya', 'Cannes', 'Kazbegi', 'Bishkek', 'Amiens',
       'Siem Reap', 'Fez', 'Colonia del Sacramento', 'Ivangorod',
       'Sao Paulo', 'Sardinia', 'Wrocław', 'Kampala', 'Toledo', 'Aachen',
       'Rethymno', 'Oslo', 'Gomel', 'Kungur', 'Puerto Madryn', 'Alicante',
       'Granada', 'Boca Chica', 'Marburg', 'Muscat', 'Rio de Janeiro',
       'Rovinj', 'Nazareth', 'test2', 'Zabljak', 'Hluboká nad Vltavou',
       'Amalfi', 'Denpasar', 'Orenburg', 'Yeysk', 'Zagreb', 'Kalyazin',
       'Bonn', 'Mumbai', 'Costa Brava', 'Volendam', 'Nizhnevartovsk',
       'Arkhangelskoye', 'Temryuk', 'Jurmala', 'Guangzhou', 'Caserta',
       'Novokuznetsk', 'Poreс', 'Kemerovo', 'Las-Palmas-de-Gran-Canaria',
       'Tyrol', 'Kurgan', 'Deauville', 'Arzamas', 'Saransk', 'Dzhubga',
       'Calabria', 'Lyon', 'Sovetsk', 'chernyakhovsk', 'Chambery',
   

### П. 3-4 Какие фильтры популярнее и как часто люди пользуются выбором цены.

In [826]:
counts_event = df.groupby(['event_action'])[['unique_events', 'total_events']].sum().reset_index()
counts_event.sort_values(by=['unique_events'], ascending=False)


,event_action,unique_events,total_events
0,Page Visit,1384478,2124993
11,search-tools-button_open,85773,94210
7,price_button_submit,63063,71890
4,filters-categories_click,62539,80065
13,ticket-type_checkbox,17918,22882
12,start_date_click,14501,19939
3,end_date_click,11738,14638
6,price_button_close,11646,14247
2,dates_filter_mobile,10272,13038
8,price_first,6676,8572


3. Считаем использование фильтров по  price_button_submit - применению фильтров
Сортировка и категории считаем использование по - filters-categories_click - сортировка и категории, активация чекбоксов
Таким образом, как мы можем видеть из этого списка по event_action:
Уникальных использований применения фильтров и использования сортировок и категорий, приблизительно равно, но всего использований больше у 
сортировок и категорий, это может быть связано с тем, что люди ищут экскурсии, используя все интересующие их параметры, а потом возвращаются и используют сортировку повторно, чтобы посмотреть сортировку по цене/популярности/рейтингу.

4. Считать выбор цены будем по price_first + price_second + price_third - уникальных использований данного фильтра 6676 + 1995 + 595 = 9266, что
   что намного меньше, чем количество использований других фильтров.

   

### Чтобы посмотреть более подробно, можно посмотреть статистику по event_action в самых крупных городах:

In [830]:
counts_page = df[df['event_action'] == 'Page Visit'].sort_values(by=['unique_events'], ascending=False)
counts_page.head(10)

,event_category,event_action,event_label,total_events,unique_events,city,text
6537,City Page,Page Visit,Saint Petersburg,350560,231941,Saint Petersburg,NaN
5428,City Page,Page Visit,Moscow,202253,134254,Moscow,NaN
13519,City Page,Page Visit,Kaliningrad,111181,67996,Kaliningrad,NaN
4912,City Page,Page Visit,Kazan,74472,51796,Kazan,NaN
7134,City Page,Page Visit,Sharm-el-Sheikh,71622,43706,Sharm-el-Sheikh,NaN
16611,City Page,Page Visit,Sochi,64054,40210,Sochi,NaN
2197,City Page,Page Visit,Istanbul,56073,37072,Istanbul,NaN
18668,City Page,Page Visit,Anapa,63348,36856,Anapa,NaN
3261,City Page,Page Visit,Hurghada,54866,32628,Hurghada,NaN
10412,City Page,Page Visit,Dubai,51377,32201,Dubai,NaN


In [832]:
counts_kaliningrad = df[df['city'] == 'Kaliningrad'].groupby(['event_action'])[['unique_events', 'total_events']].sum().reset_index()
counts_kaliningrad.sort_values(by=['unique_events'], ascending=False)

,event_action,unique_events,total_events
0,Page Visit,67996,111181
11,search-tools-button_open,6772,7740
4,filters-categories_click,5773,7824
7,price_button_submit,5222,6113
13,ticket-type_checkbox,1471,1954
12,start_date_click,1273,1824
2,dates_filter_mobile,1167,1574
3,end_date_click,1026,1354
6,price_button_close,1021,1218
8,price_first,542,748


In [834]:
counts_moscow= df[df['city'] == 'Moscow'].groupby(['event_action'])[['unique_events', 'total_events']].sum().reset_index()
counts_moscow.sort_values(by=['unique_events'], ascending=False)

,event_action,unique_events,total_events
0,Page Visit,134254,202253
11,search-tools-button_open,4384,4801
4,filters-categories_click,3292,4348
7,price_button_submit,3176,3598
2,dates_filter_mobile,940,1187
12,start_date_click,864,1155
13,ticket-type_checkbox,788,999
3,end_date_click,691,840
6,price_button_close,625,776
8,price_first,407,539


In [836]:
counts_spb = df[df['city'] == 'Saint Petersburg'].groupby(['event_action'])[['unique_events', 'total_events']].sum().reset_index()
counts_spb.sort_values(by=['unique_events'], ascending=False)

,event_action,unique_events,total_events
0,Page Visit,231941,350560
11,search-tools-button_open,6645,7323
4,filters-categories_click,5311,7482
7,price_button_submit,5028,5601
13,ticket-type_checkbox,1204,1660
12,start_date_click,1203,1637
2,dates_filter_mobile,1143,1556
3,end_date_click,958,1261
6,price_button_close,781,976
8,price_first,459,764


In [838]:
counts_kz = df[df['city'] == 'Kazan'].groupby(['event_action'])[['unique_events', 'total_events']].sum().reset_index()
counts_kz.sort_values(by=['unique_events'], ascending=False)

,event_action,unique_events,total_events
0,Page Visit,51796,74472
11,search-tools-button_open,2425,2688
4,filters-categories_click,1909,2492
7,price_button_submit,1839,2072
13,ticket-type_checkbox,466,611
12,start_date_click,399,548
3,end_date_click,314,382
2,dates_filter_mobile,296,354
6,price_button_close,292,355
5,pay-type_checkbox,184,234


Как мы можем видеть, в Москве и Санкт-Петербурге люди намного реже открывали фильтры и категории. Напротив, в Калининграде, в котором намного меньше посещений страницы города люди большее количество раз открывали их. Посмотрим как соотносится открытие фильтров и категорий к общему количеству посещений страницы города в других Городах.

In [841]:
counts1 = df[df['event_action'] == 'search-tools-button_open'].groupby('city') \
[['unique_events', 'total_events']].sum().reset_index().sort_values(by=['city'])#количество открытий фильтров
counts2 = df[(df['event_action'] == 'Page Visit') & df['city'].isin(counts1['city'])].groupby('city') \
[['unique_events', 'total_events']].sum().reset_index().sort_values(by=['city'])#количество посещений страницы города


In [843]:
division = counts1[['unique_events', 'total_events']].div(counts2[['unique_events', 'total_events']])
stats = pd.DataFrame()
stats['city'] = counts1.city
stats[['unique_filter', 'total_filter']] = counts1[['unique_events', 'total_events']]
stats[['unique_city', 'total_city']] = counts2[['unique_events', 'total_events']]
stats[['percent_unique', 'percent_total']] = division[['unique_events', 'total_events']]
stats.sort_values(by=['unique_city', 'percent_unique'], ascending=False).head(15)#показатели для самых популярных городов


,city,unique_filter,total_filter,unique_city,total_city,percent_unique,percent_total
297,Saint Petersburg,6645,7323,231941,350560,0.028650,0.020889
221,Moscow,4384,4801,134254,202253,0.032655,0.023738
149,Kaliningrad,6772,7740,67996,111181,0.099594,0.069616
156,Kazan,2425,2688,51796,74472,0.046818,0.036094
314,Sharm-el-Sheikh,2065,2263,43706,71622,0.047248,0.031596
322,Sochi,3583,3966,40210,64054,0.089107,0.061917
139,Istanbul,3208,3575,37072,56073,0.086534,0.063756
15,Anapa,2296,2506,36856,63348,0.062297,0.039559
132,Hurghada,1299,1417,32628,54866,0.039812,0.025827
91,Dubai,2694,3026,32201,51377,0.083662,0.058898


In [845]:
pd.concat([stats[stats['unique_city'] >= 1000].sort_values(by=['percent_unique'], ascending=False).head(15), \
stats[stats['unique_city'] >= 1000].sort_values(by=['percent_unique'], ascending=False).tail(15)])       
#города, в котором отношение открытия фильтров к посещению страницы города наибольшее и наименьшее(с посещением страницы города >=1000)

,city,unique_filter,total_filter,unique_city,total_city,percent_unique,percent_total
40,Belgrade,230,255,1066,1613,0.215760,0.158091
24,Ayia Napa,386,414,1881,2591,0.205210,0.159784
83,Cyprus,649,691,4197,5793,0.154634,0.119282
392,Yerevan,974,1102,6305,9524,0.154481,0.115708
59,Budapest,2358,2580,15770,24542,0.149524,0.105126
344,Tbilisi,1374,1603,9747,14979,0.140966,0.107016
390,Yaroslavl,597,664,4574,7052,0.130520,0.094158
171,Krakow,161,182,1237,1981,0.130154,0.091873
374,Vienna,322,346,2564,4017,0.125585,0.086134
33,Barcelona,264,288,2218,3246,0.119026,0.088725


### Выводы относительно использования фильтров в популярных городах
Как мы видим, действительно, во многих популярных городах люди реже пользуются фильтрами относительно всех людей, которые просматривают экскурсии 
в данном городе. Но даже среди них Москва и Санкт-Петербург - города, в которых люди пользуются фильтрами еще реже. 
Это может быть связано с тем, что экскурсии в Санкт-Петербурге и Москве чаще всего смотрят их жители и не так тщательно выбирают экскурсии по всем критериям, как при посещении города в роли туриста. Для таких пользователей может быть намного более востребованы рекомендации экскурсий на странице города, чем подробные фильтры.